In [ ]:
!pip install vecstack

from vecstack import stacking
import pandas as pd
import numpy as np


from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive/')
# from google.colab import drive
# drive.mount('/gdrive')
#Change current working directory to gdrive
# %cd /gdrive



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!find 'drive'

In [ ]:
trainfile = r'drive/MyDrive/CIS508_Workspace/Individual4/train.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'drive/MyDrive/CIS508_Workspace/Individual4/test.csv'
testData = pd.read_csv(testfile)  #creates a dataframe


print(trainData.shape)
print(testData.shape)



(137, 43)
(100000, 42)


In [ ]:
# trainData.isna().sum()
trainData.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3.0,5,1,3,2,3,4,3,3,4316715.0


In [ ]:
#To get list of names of all Columns from a dataframe

TrainCols = list(trainData.columns.values)
TestCols = list(testData.columns.values)
print(TrainCols)
print(TestCols)

['Id', 'Open Date', 'City', 'City Group', 'Type', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37', 'revenue']
['Id', 'Open Date', 'City', 'City Group', 'Type', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37']


In [ ]:
#Extract training and test data
y_train = trainData["revenue"]
X_train = trainData.drop(["revenue"], axis=1) #extracting training data without the target column
X_test = testData #extracting training data without the target column

print(X_train.shape)
print(X_test.shape)


(137, 42)
(100000, 42)


In [ ]:
list(trainData.select_dtypes(include=['object']).columns.values)

['Open Date', 'City', 'City Group', 'Type']

In [ ]:
categoricalFeatures=['Open Date', 'City', 'City Group', 'Type']

In [ ]:
# OneHotEncoding on Train (fit & transform)
# OneHotEncoding is to be done on Categorical variables.
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
Xcat = pd.DataFrame(ohe.fit_transform(X_train[categoricalFeatures]),columns=ohe.get_feature_names_out(),index=X_train.index)
X_train = pd.concat([X_train,Xcat],axis=1)
X_train.drop(labels=categoricalFeatures,axis=1,inplace=True)
X_train.sample(5)

,Id,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,City_Trabzon,City_Uşak,City_İstanbul,City_İzmir,City_Şanlıurfa,City Group_Big Cities,City Group_Other,Type_DT,Type_FC,Type_IL
115,115,6,4.5,6.0,6.0,4,8,10,10,10,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,5,6,6.0,4.5,7.5,8,10,10,8,8,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
17,17,2,4.0,4.0,5.0,1,3,5,4,5,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
53,53,3,4.0,4.0,3.0,1,2,5,5,5,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
42,42,12,7.5,6.0,6.0,4,4,10,8,10,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
#test OneHotEncoding
# OneHotEncoding on Test (only transform)
# OneHotEncoding is to be done on Categorical variables.
Xcat = pd.DataFrame(ohe.transform(X_test[categoricalFeatures]),columns=ohe.get_feature_names_out(),index=X_test.index)
X_test = pd.concat([X_test,Xcat],axis=1)
X_test.drop(labels=categoricalFeatures,axis=1,inplace=True)
X_test.sample(5)

In [ ]:
X_test.head()

,Id,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,City_Çorum,City_İstanbul,City_İzmir,City_Şanlıurfa,City Group_Big Cities,City Group_Other,Type_DT,Type_FC,Type_IL,Type_MB
0,0,1,4.0,4.0,4.0,1,2,5,4,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1,3,4.0,4.0,4.0,2,2,5,3,4,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,2,3,4.0,4.0,4.0,2,2,5,4,4,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,3,2,4.0,4.0,4.0,2,3,5,4,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,4,2,4.0,4.0,4.0,1,2,5,4,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [ ]:
#Decision Tree Regressor ========================================================================
#DECISION TREE with hyperparameter tuning
dtr = DecisionTreeRegressor()
# Hyper Parameter Tuning
dtr_params={'criterion':['squared_error','friedman_mse','absolute_error','poisson','entropy'],
        'splitter':['best','random'],
       'max_depth':range(2,20,2),
       'max_leaf_nodes': range(5,50,5)}

rscv=RandomizedSearchCV(dtr,dtr_params)
rscv.fit(X_train,y_train)
dtr_best_parameters=rscv.best_params_
print('The Best Paramteres for Decision Tree Regressor is : ',rscv.best_params_)
# print('\nThe Best Score for Decision Tree Regressor is : ',rscv.best_score_)

dtr=DecisionTreeRegressor(**dtr_best_parameters)
dtr.fit(X_train,y_train)
DTR_Prediction=dtr.predict(X_test)

The Best Paramteres for Decision Tree Regressor is :  {'splitter': 'best', 'max_leaf_nodes': 40, 'max_depth': 2, 'criterion': 'absolute_error'}

The Best Score for Decision Tree Regressor is :  -0.198800440929375


In [ ]:
#Decision Tree predictions
DTR_Prediction = pd.DataFrame (DTR_Prediction, columns=[ 'Prediction' ])
DTR_Prediction[ 'Id'] = testData[ 'Id']
DTR_Prediction.tail ()

,Prediction,Id
99995,4084226.0,99995
99996,2739369.5,99996
99997,4084226.0,99997
99998,4084226.0,99998
99999,4084226.0,99999


In [ ]:
#DTR Predictions Save to CSV file
DTR_Prediction.to_csv('DTR_WS.csv', index=False)

In [ ]:
#Random Forest Regressor==============================================================================
#=================================================================================================
# Model
rfc = RandomForestRegressor()

#Hyperparameter Tuning & K-fold Cross Validation
# Doing HyperParameter Tuning & K-fold Cross Validation on RandomForestRegressor
parameters={'min_samples_leaf' : range(20,100,20),
               'max_depth': range(2,10,1),
               'max_features':[10,20,30,40,50],
               'n_estimators':[20,30,40]}

# After doing Hyperparameter Tuning,Using the RandomForestRegressor with the best parameters
rf_random = RandomizedSearchCV(rfc,parameters,n_iter=25,cv=5)
rf_random.fit(X_train, y_train)
grid_parm=rf_random.best_params_
print('The Best Paramteres for Random Forest Regressor are : ',grid_parm)
print('\nThe Best Score for Random Forest Regressor is : ',rf_random.best_score_)

#Use best parameter
rffinal = RandomForestRegressor(**grid_parm)
rffinal.fit(X_train, y_train)
rfc_predict=rffinal.predict(X_test)

The Best Paramteres for Random Forest Regressor are :  {'n_estimators': 40, 'min_samples_leaf': 20, 'max_features': 40, 'max_depth': 4}

The Best Score for Random Forest Regressor is :  0.03504423760852984


In [ ]:
#Random forest predictions
rfc_predict = pd.DataFrame (rfc_predict, columns=[ 'Prediction' ])
rfc_predict[ 'Id'] = testData[ 'Id']
rfc_predict.tail ()

,Prediction,Id
99995,4.403007e+06,99995
99996,3.688769e+06,99996
99997,5.107734e+06,99997
99998,5.230336e+06,99998
99999,4.764213e+06,99999


In [ ]:
#Random forest Save to CSV file
rfc_predict.to_csv('RFC_WS.csv', index=False)

In [ ]:
# Support Vector Regressor
from sklearn.svm import LinearSVR
regressor = LinearSVR()

parameters = {'loss':['epsilon_insensitive', 'squared_epsilon_insensitive'],
              'max_iter':range(100,1500,200)}

randomsearch = RandomizedSearchCV(regressor, parameters)
randomsearch.fit(X_train, y_train)
best = randomsearch.best_params_
print('The Best Parameters for LinearSVR are : ',best)
print('\nThe Best Score for LinearSVR are : ',randomsearch.best_score_)

#Use the best parameters
LSVR_pred = LinearSVR(**best)
LSVR_pred.fit(X_train, y_train)
# svm_predict = pd.DataFrame(regressor45552.predict(X_test), columns = ['Prediction'])

The Best Parameters for LinearSVR are :  {'max_iter': 700, 'loss': 'squared_epsilon_insensitive'}

The Best Score for LinearSVR are :  -1.9524498073201901


LinearSVR(loss='squared_epsilon_insensitive', max_iter=700)

In [ ]:
# Predictions of LSVR
best_LSVR_model = LinearSVR(**best_params)
best_LSVR_model.fit(X_train, y_train)

# Make predictions
LSVR_predictions = best_LSVR_model.predict(X_test)

# Create DataFrame with predictions and 'Id' column
LSVR_pred = pd.DataFrame({'Prediction': LSVR_predictions, 'Id': testData['Id']})

# Display the tail of the DataFrame
print(LSVR_pred.tail())

# Save predictions to a CSV file
LSVR_pred.to_csv('LSVR_WS2.csv', index=False)

         Prediction     Id
99995  2.119195e+08  99995
99996  2.113596e+08  99996
99997  2.134294e+08  99997
99998  2.154150e+08  99998
99999  2.164332e+08  99999


In [ ]:
# GradientBoostingRegressor
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

gb = GradientBoostingRegressor()

#GridSearchCV
grid_search = GridSearchCV(gb, param_grid, cv=4, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

#best hyperparameters
best_gb_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best hyperparameters for GradientBoostingRegressor:", best_gb_model)

# Use the best hyperparameters to train the model
best_gb_model.fit(X_train, y_train)

# GB predictions
gb_predictions_test = best_gb_model.predict(X_test)




Best hyperparameters for GradientBoostingRegressor: GradientBoostingRegressor(learning_rate=0.01, n_estimators=50)


In [ ]:
# GB Predictions
gb_predictions_test = pd.DataFrame (gb_predictions_test, columns=[ 'Prediction' ])
gb_predictions_test[ 'Id'] = testData[ 'Id']
gb_predictions_test.tail ()

,Prediction,Id
99995,4.379836e+06,99995
99996,4.272622e+06,99996
99997,4.403075e+06,99997
99998,4.403075e+06,99998
99999,4.403075e+06,99999


In [ ]:
# GB Predictions save to csv
gb_predictions_test.to_csv('GB_WS.csv', index=False)

In [ ]:
#MLP regressor
mlp = MLPRegressor()
mlp.fit(X_train, y_train)

# RandomCV Hyper Parameter Tuning
mlp_parameters = {'hidden_layer_sizes': [(50, 50), (100, 100), (50, 100, 50)],
                  'activation': ['relu', 'tanh', 'logistic'],
                  'alpha': [0.0001, 0.001, 0.01]}
mlp_random = RandomizedSearchCV(mlp, mlp_parameters, n_iter=15)
mlp_random.fit(X_train, y_train)
print("Best set of parameters for mlp: ", mlp_random.best_params_)
mlp_best_params = mlp_random.best_params_

mlp_tuned = MLPRegressor(**mlp_best_params)
mlp_tuned.fit(X_train, y_train)

# Accuracy on the training set after HyperParameterTuning
train_accuracy_mlp_tuned = mlp_tuned.score(X_train, y_train)
print("\nAccuracy Score (Training set): {:.6f}".format(train_accuracy_mlp_tuned))

# Predicted values for the test set
MLP_Predictions = mlp_tuned.predict(X_test)
print("\nPredicted values (Test Set):")
print(MLP_Predictions)

Best set of parameters for mlp:  {'hidden_layer_sizes': (50, 100, 50), 'alpha': 0.01, 'activation': 'relu'}

Accuracy Score (Training set): -2.708436

Predicted values (Test Set):
[9.58126312e+04 9.05786832e+04 1.11951647e+05 ... 1.23298235e+08
 1.23348124e+08 1.23250794e+08]


In [ ]:
# mlp Predictions
mlp_predictions_df = pd.DataFrame({'Prediction': MLP_Predictions, 'Id': testData['Id']})
print(mlp_predictions_df.tail())

# Save predictions to a CSV file
mlp_predictions_df.to_csv('mlp_WS.csv', index=False)


         Prediction     Id
99995  1.232084e+08  99995
99996  1.231974e+08  99996
99997  1.232982e+08  99997
99998  1.233481e+08  99998
99999  1.232508e+08  99999


In [ ]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingRegressor(), RandomForestRegressor(), DecisionTreeRegressor() ]

S_Train, S_Test = stacking( models,
                           X_train, y_train, X_test,
                           regression=True,

                           mode='oof_pred_bag',

                           needs_proba=False,

                           save_dir=None,

                           n_folds=4,
                           shuffle=True,
                           verbose=2)



___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingRegressor]
    fold  0:  [1653639.52267378]
    fold  1:  [1295635.02387851]
    fold  2:  [1868366.49586036]
    fold  3:  [1760364.12849053]
    ----
    MEAN:     [1644501.29272579] + [215250.43941415]
    FULL:     [1644567.99513417]

model  1:     [RandomForestRegressor]
    fold  0:  [1595846.43971429]
    fold  1:  [1199347.96823529]
    fold  2:  [1825113.02558824]
    fold  3:  [1637819.95000000]
    ----
    MEAN:     [1564531.84588445] + [227823.15822929]
    FULL:     [1564760.41956204]

model  2:     [DecisionTreeRegressor]
    fold  0:  [1914909.34285714]
    fold  1:  [1345625.23529412]
    fold  2:  [1957084.11764706]
    fold  3:  [1866260.29411

In [ ]:
#GradientBoostingRegressor with stacking
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# Define the parameters grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

stacked_model = GradientBoostingRegressor()

# GridSearchCV
grid_search = GridSearchCV(stacked_model, param_grid, cv=4, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(S_Train, y_train)

# best parameters
best_params = grid_search.best_params_
print("Best hyperparameters for GradientBoostong:", best_params)
# Train the stacked model with the best parameters
finalGB_stacked_model = GradientBoostingRegressor(**best_params)
finalGB_stacked_model.fit(S_Train, y_train)

# Corrected prediction
GB_pred = finalGB_stacked_model.predict(S_Test)

Best hyperparameters for GradientBoostong: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}


In [ ]:
#GB Predictions
GB_pred = pd.DataFrame (GB_pred, columns=[ 'Prediction' ])
GB_pred[ 'Id'] = testData[ 'Id']
GB_pred.tail ()

,Prediction,Id
99995,5.030562e+06,99995
99996,4.292057e+06,99996
99997,5.030562e+06,99997
99998,5.030562e+06,99998
99999,5.030562e+06,99999


In [ ]:
# GB Predictions save to csv
GB_pred.to_csv('GBResultfinalout2.csv', index=False)

In [ ]:
# RandomForestRegressor with stacking
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestRegressor()

# Perform GridSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=4, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(S_Train, y_train)

#best hyperparameters
best_rf_model = grid_search.best_estimator_

# # Print the best hyperparameters
print("Best hyperparameters for RandomForestRegressor:", best_rf_model)
# Train the stacked model with the best parameters
finalRF_stacked_model = RandomForestRegressor(
    n_estimators=best_rf_model.n_estimators,
    max_depth=best_rf_model.max_depth,
    min_samples_split=best_rf_model.min_samples_split,
    min_samples_leaf=best_rf_model.min_samples_leaf
)
finalRF_stacked_model.fit(S_Train, y_train)
RF_pred=finalRF_stacked_model.predict(S_Test)

Best hyperparameters for RandomForestRegressor: RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=5,
                      n_estimators=50)


In [ ]:
# RF stacked Predictions
RF_pred = pd.DataFrame (RF_pred, columns=[ 'Prediction' ])
RF_pred[ 'Id'] = testData[ 'Id']
RF_pred.tail ()

,Prediction,Id
99995,5.747221e+06,99995
99996,4.966939e+06,99996
99997,6.618564e+06,99997
99998,6.491855e+06,99998
99999,6.618564e+06,99999


In [ ]:
#RF save to CSV
RF_pred.to_csv('RFResultfinalout.csv', index=False)

In [ ]:
#DecisionTreeRegressor with stacking
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

# parameter grid
param_grid = {
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt = DecisionTreeRegressor()

# Perform GridSearchCV
grid_search = GridSearchCV(dt, param_grid, cv=4, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(S_Train, y_train)

#best hyperparameters
best_dt_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best hyperparameters for DecisionTreeRegressor:", best_dt_model)
# Train the stacked model with the best parameters
finalDT_stacked_model = DecisionTreeRegressor(
    max_depth=best_dt_model.max_depth,
    min_samples_split=best_dt_model.min_samples_split,
    min_samples_leaf=best_dt_model.min_samples_leaf
)
finalDT_stacked_model.fit(S_Train, y_train)
DT_pred=finalDT_stacked_model.predict(S_Test)


Best hyperparameters for DecisionTreeRegressor: DecisionTreeRegressor(max_depth=5, min_samples_leaf=4, min_samples_split=10)


In [ ]:
#DT Predictions
DT_pred = pd.DataFrame (DT_pred, columns=[ 'Prediction' ])
DT_pred[ 'Id'] = testData[ 'Id']
DT_pred.tail ()

,Prediction,Id
99995,4.907477e+06,99995
99996,3.309795e+06,99996
99997,8.190165e+06,99997
99998,8.190165e+06,99998
99999,8.190165e+06,99999


In [ ]:
# DT pred Save to CSV
DT_pred.to_csv('DTResultfinalout.csv', index=False)